In [40]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from astropy.cosmology import FlatLambdaCDM
from NFW.nfw import NFW
from NFW import mass_concentration

from colossus.cosmology import cosmology as colossuscosmo
from colossus.halo import concentration

%matplotlib inline
mpl.rcParams.update(mpl.rcParamsDefault)

from ipywidgets import interactive

In [5]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
params = {'flat': True, 'H0': cosmo.H0.value,
          'Om0': cosmo.Om0, 'Ob0': 0.046,
          'sigma8': 0.81, 'ns': 0.95}
colossus_cosmo = colossuscosmo.setCosmology('myCosmo', params)

In [14]:
z = 0.6
m200 = 8e14
c200dk = concentration.concentration(m200 / params['H0'] / 100, '200c', z)
print(f"DK15 concentration for a halo of M200c {m200:.2e} at redshift {z}: {c200dk:.3}")
c200duffy = mass_concentration.duffy_concentration(m200, z, cosmo)
print(f"Duffy08+ concentration for a halo of M200c {m200:.2e} at redshift {z}: {c200duffy:.3}")

DK15 concentration for a halo of M200c 8.00e+14 at redshift 0.6: 7.01
Duffy08+ concentration for a halo of M200c 8.00e+14 at redshift 0.6: 2.85


In [36]:
def likeli(m200, z, cosmo, r, delta_sigma):
    c200 = mass_concentration.duffy_concentration(m200, z, cosmo)
    nfw = NFW(m200, c200, z, cosmology=cosmo)
    delta_sigma_model = nfw.delta_sigma(r)
    return np.sum((delta_sigma.value - delta_sigma_model.value)**2)

In [69]:
def interactive_fitting(m200, z, inner_radius):
    c200dk = concentration.concentration(m200 / params['H0'] / 100, '200c', z)
    nfw = NFW(m200, c200, z, cosmology=cosmo)
    r = np.logspace(np.log10(inner_radius), 0.4)
    delta_sigma = nfw.delta_sigma(r)
    res = minimize(likeli, 5e14, args=(z, cosmo, r, delta_sigma), method='Nelder-Mead')
    m200fit = res.x[0]
    m_err = (m200fit - m200) / m200
    
    c200duffy = mass_concentration.duffy_concentration(m200fit, z, cosmo)
    nfw_duffy = NFW(m200fit, c200duffy, z, cosmology=cosmo)
    
    r_plot = np.logspace(-1, 0.4)
    delta_sigma_true = nfw.delta_sigma(r_plot)
    delta_sigma_model = nfw_duffy.delta_sigma(r_plot)
    plt.loglog(r_plot, delta_sigma_true, label='Input (DK15)')
    plt.loglog(r_plot, delta_sigma_model, label='Best Fit (Duffy08)')
    #plt.plot(r_plot, delta_sigma_model - delta_sigma_true)
    plt.legend()
    plt.xlabel(r"$r/\mathrm{Mpc}$")
    plt.ylabel("$\Delta\Sigma / M_\odot \mathrm{Mpc}^2$")
    plt.title(f"Fractional mass error: {m_err:.3}")
    plt.show()
    

In [72]:
interactive_plot = interactive(interactive_fitting, m200=(1e14, 2e15),
                               z=(0.3, 1.2),
                               inner_radius=(0.1, 1.5))
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=1050000000000000.0, description='m200', max=2000000000000000.0, min=10…